In [13]:
from multiprocessing import Pool , cpu_count
from time import time

def square(x):
    [x for x in range(10000)]
    return x ** 2

n_cpu = cpu_count()
print(f'n_cpu = {n_cpu}')
p = Pool(n_cpu-1)

start = time()
for s in p.imap(square , range(900)):
    print(f'\rs = {s}', end ='')
end = time()
print('\ntime = {}'.format(end-start))

In [ ]:
# 데이터를 병렬으로 불러오기 위한 function
def parallelize_dataframe(df, func , num_cores):
    print('프로세스 개수 :' , num_cores)
    print('프로세스 당 데이터 개수 :' , len(df) // num_cores)
    # 데이터 나누고, 쓰레드 할당받기
    df_split = np.array_split(df, num_cores)
    pool = Pool(num_cores)
    result = pool.map(func, df_split)
    pool.close()
    pool.join()

    # 결과 합치기
    result_1 , result_2 = [] , []
    for i in result:
        result_1.append(i[0])
        result_2.append(i[1])
    df_result = pd.concat(result_1 , axis = 0 )
    df_item = pd.concat(result_2 , axis = 0 )

    return df_result , df_item

# json to df
def json_loader(data_path):
    df = pd.DataFrame()
    df_item = pd.DataFrame()
    df_error = []

    for i ,data_i in enumerate(data_path):
        with open(data_i, 'r') as json_read :
            json_str = json.load(json_read) # json.load 로 파일 읽기
            json_data = json_str['FILE'][0]
            # json file path to img file path
            data_i = data_i.replace('json', 'jpg')
            data_i = data_i.replace('label', 'img')
            json_data['FILE_NAME'] = data_i
            phase = 'train' if 'train' in data_i else 'val'
            json_data['type'] = phase
            # get img info
            df = df.append( json_data, ignore_index= True)
            for json_item in json_data['ITEMS']:
                json_item['FILE_NAME'] = json_data['FILE_NAME']
                json_item['type'] = phase

                # get obj info
                df_item = df_item.append(json_item , ignore_index= True)
                
    return df , df_item


df , df_item = parallelize_dataframe(data_all_path , json_loader , mp.cpu_count() // 2)